In [256]:
# 이 프로젝트는 [Kaggle Groceries Dataset](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset) (GPL v2 라이선스) 기반으로 분석되었습니다.
# 이 분석 결과 및 파생 데이터/코드는 GPL v2 라이선스의 조건을 따릅니다.

In [257]:
import pandas as pd
import os 

base_dir = os.path.abspath(os.path.dirname(os.path.join(os.getcwd(), "Association_Rules_Groceries.ipynb")))
file_path = os.path.join(base_dir, "..", "data", "Groceries_dataset.csv")
df = pd.read_csv(file_path)
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [258]:
# Date 컬럼을 datetime 형식으로 변환 ( '21-07-2015' = '%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# 연도, 월, 일, 요일 컬럼 추가
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()   # (ex. Monday, Tuesday)

df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday
0,1808,2015-07-21,tropical fruit,2015,7,21,Tuesday
1,2552,2015-01-05,whole milk,2015,1,5,Monday
2,2300,2015-09-19,pip fruit,2015,9,19,Saturday
3,1187,2015-12-12,other vegetables,2015,12,12,Saturday
4,3037,2015-02-01,whole milk,2015,2,1,Sunday


In [259]:
# 계절(Season) 컬럼 추가 함수

def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    elif month in [12, 1, 2]:
        return 'Winter'


df['Season'] = df['Month'].apply(get_season)

df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season
0,1808,2015-07-21,tropical fruit,2015,7,21,Tuesday,Summer
1,2552,2015-01-05,whole milk,2015,1,5,Monday,Winter
2,2300,2015-09-19,pip fruit,2015,9,19,Saturday,Autumn
3,1187,2015-12-12,other vegetables,2015,12,12,Saturday,Winter
4,3037,2015-02-01,whole milk,2015,2,1,Sunday,Winter


In [260]:
df['YearMonth'] = df['Date'].dt.to_period('M')
df['WeekOfYear'] = df['Date'].dt.isocalendar().week
df['Quarter'] = df['Date'].dt.quarter
df['IsMonthStart'] = df['Date'].dt.is_month_start
df['IsMonthEnd'] = df['Date'].dt.is_month_end
df['IsWeekend'] = df['Date'].dt.weekday >= 5
df['DayOfYear'] = df['Date'].dt.dayofyear

df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear
0,1808,2015-07-21,tropical fruit,2015,7,21,Tuesday,Summer,2015-07,30,3,False,False,False,202
1,2552,2015-01-05,whole milk,2015,1,5,Monday,Winter,2015-01,2,1,False,False,False,5
2,2300,2015-09-19,pip fruit,2015,9,19,Saturday,Autumn,2015-09,38,3,False,False,True,262
3,1187,2015-12-12,other vegetables,2015,12,12,Saturday,Winter,2015-12,50,4,False,False,True,346
4,3037,2015-02-01,whole milk,2015,2,1,Sunday,Winter,2015-02,5,1,True,False,True,32


In [261]:
years = df['Date'].dt.year.unique()
print(years)

[2015 2014]


In [262]:
# 데이터 셋을 올린 Heeral Dedhia 가 인도사람이기에 인도의 holiday로 가정

import pandas as pd
import holidays

# 날짜 파싱
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# 인도 공휴일 객체 만들기 (2014, 2015년만)
indian_holidays = holidays.India(years=[2014, 2015])

# 휴일 여부 플래그 생성
df['IsHoliday'] = df['Date'].isin(indian_holidays)

# 결과 확인
print(df[['Date', 'IsHoliday']].head())

        Date  IsHoliday
0 2015-07-21      False
1 2015-01-05      False
2 2015-09-19      False
3 2015-12-12      False
4 2015-02-01      False


C:\Users\user\AppData\Local\Temp\ipykernel_3236\305225185.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['IsHoliday'] = df['Date'].isin(indian_holidays)


In [263]:
df = df.sort_values(['Member_number', 'Date'])
df['PrevPurchaseDelta'] = df.groupby('Member_number')['Date'].diff().dt.days

df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear,IsHoliday,PrevPurchaseDelta
13331,1000,2014-06-24,whole milk,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,NaN
29480,1000,2014-06-24,pastry,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0
32851,1000,2014-06-24,salty snack,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0
4843,1000,2015-03-15,sausage,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,264.0
8395,1000,2015-03-15,whole milk,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,0.0


In [264]:
n_unique = df['itemDescription'].nunique()
print(f"itemDescription의 유니크 개수: {n_unique}")

itemDescription의 유니크 개수: 167


In [265]:
unique_items = df['itemDescription'].unique().tolist()
print(unique_items)

['whole milk', 'pastry', 'salty snack', 'sausage', 'semi-finished bread', 'yogurt', 'soda', 'pickled vegetables', 'canned beer', 'misc. beverages', 'hygiene articles', 'rolls/buns', 'frankfurter', 'whipped/sour cream', 'beef', 'white bread', 'curd', 'frozen vegetables', 'other vegetables', 'butter', 'tropical fruit', 'sugar', 'butter milk', 'specialty chocolate', 'frozen meals', 'dental care', 'root vegetables', 'detergent', 'pip fruit', 'dish cleaner', 'cling film/bags', 'red/blush wine', 'chocolate', 'frozen fish', 'shopping bags', 'packaged fruit/vegetables', 'margarine', 'rice', 'bottled water', 'softener', 'skin care', 'chicken', 'flour', 'bottled beer', 'domestic eggs', 'white wine', 'photo/film', 'hamburger meat', 'liquor (appetizer)', 'liver loaf', 'dessert', 'canned fish', 'cocoa drinks', 'ketchup', 'herbs', 'newspapers', 'coffee', 'UHT-milk', 'kitchen towels', 'sliced cheese', 'specialty bar', 'candles', 'citrus fruit', 'candy', 'curd cheese', 'grapes', 'brown bread', 'proces

In [266]:
def map_category(item):
    item = item.lower().strip()
    if any(x in item for x in ['milk', 'cheese', 'cream', 'butter', 'yogurt', 'curd']):
        return 'Dairy'
    elif any(x in item for x in ['bread', 'roll', 'cake', 'waffle', 'bakery', 'zwieback']):
        return 'Bakery'
    elif any(x in item for x in ['beef', 'pork', 'chicken', 'sausage', 'ham', 'turkey', 'meat', 'liver loaf', 'liver', 'frankfurter']):
        return 'Meat'
    elif any(x in item for x in ['fruit', 'vegetable', 'onion', 'herbs', 'grapes', 'berries', 'potato', 'citrus']):
        return 'Produce'
    elif any(x in item for x in ['fish', 'canned fish']):
        return 'Fish'
    elif any(x in item for x in ['beer', 'wine', 'liquor', 'whisky', 'brandy', 'prosecco', 'rum', 'liqueur', 'cocoa', 'soda', 'beverages', 'juice']):
        return 'Drinks'
    elif any(x in item for x in ['dog food', 'cat food', 'pet care']):
        return 'Pet'
    elif any(x in item for x in ['snack', 'candy', 'chocolate', 'waffle', 'cake bar', 'nut']):
        return 'Snacks'
    elif any(x in item for x in ['egg']):
        return 'Eggs'
    elif any(x in item for x in ['rice', 'pasta', 'noodle', 'cereal']):
        return 'Staples'
    elif any(x in item for x in ['oil', 'vinegar', 'mustard', 'mayonnaise', 'sauce', 'ketchup']):
        return 'Condiments'
    elif any(x in item for x in ['detergent', 'cleaner', 'softener', 'house keeping', 'bathroom cleaner', 'toilet cleaner', 'abrasive']):
        return 'Household'
    elif any(x in item for x in ['make up', 'cosmetic', 'decalcifier', 'skin care', 'hygiene', 'soap', 'napkin']):
        return 'PersonalCare'
    elif any(x in item for x in ['flower', 'plant', 'soil', 'pot']):
        return 'Flowers'
    elif any(x in item for x in ['shopping bags', 'bags']):
        return 'Bag'
    elif any(x in item for x in ['pastry', 'dessert', 'sweet', 'jam', 'pudding', 'soups', 'soup', 'popcorn', 'honey', 'instant', 'frozen dessert']):
        return 'ProcessedFood'
    elif any(x in item for x in ['dental care', 'female sanitary', 'hair spray', 'kitchen towels', 'kitchen utensil']):
        return 'PersonalCare'
    elif any(x in item for x in ['photo', 'film', 'light bulb']):
        return 'Household'
    elif any(x in item for x in ['candles']):
        return 'Household'
    elif any(x in item for x in ['newspapers']):
        return 'Media'
    elif any(x in item for x in ['cookware', 'dishes', 'preservation products']):
        return 'Kitchenware'
    elif any(x in item for x in ['artif. sweetener', 'baking powder', 'spices', 'salt']):
        return 'SpicesCondiments'
    elif any(x in item for x in ['organic products']):
        return 'Organic'
    elif any(x in item for x in ['salad dressing']):
        return 'Condiments'
    elif any(x in item for x in ['rubbing alcohol']):
        return 'Medicine'
    elif item in ['sugar', 'flour', 'syrup']:
        return 'Baking'
    elif item in ['frozen meals', 'finished products']:
        return 'ProcessedFood'
    elif item in ['margarine', 'specialty fat']:
        return 'Dairy'
    elif item in ['bottled water', 'coffee', 'tea']:
        return 'Drinks'
    elif item in ['specialty bar', 'chewing gum', 'tidbits']:
        return 'Snacks'
    elif item in ['seasonal products']:
        return 'Seasonal'
    else:
        return 'Other'

In [267]:
df['ItemCategory'] = df['itemDescription'].apply(map_category)
print(df['ItemCategory'].value_counts())
print(df[df['ItemCategory']=='Other']['itemDescription'].unique())

ItemCategory
Dairy               8301
Produce             7993
Drinks              5514
Meat                4037
Bakery              3575
ProcessedFood       1943
Snacks              1669
Bag                  809
PersonalCare         750
Media                596
Eggs                 566
Condiments           546
Household            533
Baking               432
Pet                  329
SpicesCondiments     251
Fish                 247
Staples              212
Flowers              188
Kitchenware          153
Seasonal             106
Organic               10
Medicine               5
Name: count, dtype: int64
[]


In [268]:
df['ItemCategory'] = df['itemDescription'].apply(map_category)

In [269]:
print(df[['itemDescription', 'ItemCategory']].head(20))
print(df['ItemCategory'].value_counts())

           itemDescription   ItemCategory
13331           whole milk          Dairy
29480               pastry  ProcessedFood
32851          salty snack         Snacks
4843               sausage           Meat
8395            whole milk          Dairy
20992  semi-finished bread         Bakery
24544               yogurt          Dairy
1629                  soda         Drinks
17778   pickled vegetables        Produce
2047           canned beer         Drinks
18196      misc. beverages         Drinks
6388               sausage           Meat
22537     hygiene articles   PersonalCare
9391               sausage           Meat
25540           whole milk          Dairy
32727           rolls/buns         Bakery
11046           whole milk          Dairy
27195                 soda         Drinks
364            frankfurter           Meat
16513                 soda         Drinks
ItemCategory
Dairy               8301
Produce             7993
Drinks              5514
Meat                4037
Baker

In [270]:
df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear,IsHoliday,PrevPurchaseDelta,ItemCategory
13331,1000,2014-06-24,whole milk,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,NaN,Dairy
29480,1000,2014-06-24,pastry,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,ProcessedFood
32851,1000,2014-06-24,salty snack,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,Snacks
4843,1000,2015-03-15,sausage,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,264.0,Meat
8395,1000,2015-03-15,whole milk,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,0.0,Dairy


In [271]:
# 거래별(트랜잭션별) 카테고리 개수
category_count = df.groupby(['Member_number', 'Date'])['ItemCategory'].nunique().reset_index(name='NumCategories')
df = df.merge(category_count, on=['Member_number', 'Date'])
df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear,IsHoliday,PrevPurchaseDelta,ItemCategory,NumCategories
0,1000,2014-06-24,whole milk,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,NaN,Dairy,3
1,1000,2014-06-24,pastry,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,ProcessedFood,3
2,1000,2014-06-24,salty snack,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,Snacks,3
3,1000,2015-03-15,sausage,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,264.0,Meat,3
4,1000,2015-03-15,whole milk,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,0.0,Dairy,3


In [272]:
# 거래별 대표 카테고리/최다 카테고리

def get_mode_category(group):
    return group['ItemCategory'].mode()[0]

mode_cat = df.groupby(['Member_number', 'Date']).apply(get_mode_category).reset_index(name='MainCategory')
df = df.merge(mode_cat, on=['Member_number', 'Date'])

df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_3236\3338024650.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mode_cat = df.groupby(['Member_number', 'Date']).apply(get_mode_category).reset_index(name='MainCategory')


,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear,IsHoliday,PrevPurchaseDelta,ItemCategory,NumCategories,MainCategory
0,1000,2014-06-24,whole milk,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,NaN,Dairy,3,Dairy
1,1000,2014-06-24,pastry,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,ProcessedFood,3,Dairy
2,1000,2014-06-24,salty snack,2014,6,24,Tuesday,Summer,2014-06,26,2,False,False,False,175,False,0.0,Snacks,3,Dairy
3,1000,2015-03-15,sausage,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,264.0,Meat,3,Dairy
4,1000,2015-03-15,whole milk,2015,3,15,Sunday,Spring,2015-03,11,1,False,False,True,74,False,0.0,Dairy,3,Dairy


In [273]:
# 카테고리별 재구매주기 피처 생성
df['Date'] = pd.to_datetime(df['Date'])

# 고객-카테고리별 구매 날짜만 추출 (중복 제거)
df_cat = df[['Member_number', 'ItemCategory', 'Date']].drop_duplicates()

# 고객-카테고리별로 구매 날짜를 정렬 후, 날짜 차이 계산
df_cat['CategoryPurchaseDelta'] = df_cat.sort_values('Date').groupby(['Member_number', 'ItemCategory'])['Date'].diff().dt.days

# 원래 데이터와 합치고 싶으면 merge
df = df.merge(df_cat[['Member_number', 'ItemCategory', 'Date', 'CategoryPurchaseDelta']],
            on=['Member_number', 'ItemCategory', 'Date'],
            how='left')

df.head()

,Member_number,Date,itemDescription,Year,Month,Day,Weekday,Season,YearMonth,WeekOfYear,...,IsMonthStart,IsMonthEnd,IsWeekend,DayOfYear,IsHoliday,PrevPurchaseDelta,ItemCategory,NumCategories,MainCategory,CategoryPurchaseDelta
0,1000,2014-06-24,whole milk,2014,6,24,Tuesday,Summer,2014-06,26,...,False,False,False,175,False,NaN,Dairy,3,Dairy,NaN
1,1000,2014-06-24,pastry,2014,6,24,Tuesday,Summer,2014-06,26,...,False,False,False,175,False,0.0,ProcessedFood,3,Dairy,NaN
2,1000,2014-06-24,salty snack,2014,6,24,Tuesday,Summer,2014-06,26,...,False,False,False,175,False,0.0,Snacks,3,Dairy,NaN
3,1000,2015-03-15,sausage,2015,3,15,Sunday,Spring,2015-03,11,...,False,False,True,74,False,264.0,Meat,3,Dairy,NaN
4,1000,2015-03-15,whole milk,2015,3,15,Sunday,Spring,2015-03,11,...,False,False,True,74,False,0.0,Dairy,3,Dairy,264.0


In [274]:
# 품목 단위(아이템 단위) 연관분석
transactions_item = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).tolist()


In [275]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions_item).transform(transactions_item)
df_matrix = pd.DataFrame(te_ary, columns=te.columns_)
df_matrix

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [276]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(df_matrix, min_support=0.001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)
...,...,...
745,0.001136,"(rolls/buns, whole milk, sausage)"
746,0.001002,"(rolls/buns, whole milk, soda)"
747,0.001337,"(rolls/buns, whole milk, yogurt)"
748,0.001069,"(whole milk, sausage, soda)"


In [277]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(bottled water),(UHT-milk),0.060683,0.021386,0.001069,0.017621,0.823954,1.0,-0.000228,0.996168,-0.185312,0.013201,-0.003847,0.033811
1,(UHT-milk),(bottled water),0.021386,0.060683,0.001069,0.050000,0.823954,1.0,-0.000228,0.988755,-0.179204,0.013201,-0.011373,0.033811
2,(UHT-milk),(other vegetables),0.021386,0.122101,0.002139,0.100000,0.818993,1.0,-0.000473,0.975443,-0.184234,0.015130,-0.025175,0.058758
3,(other vegetables),(UHT-milk),0.122101,0.021386,0.002139,0.017515,0.818993,1.0,-0.000473,0.996060,-0.201119,0.015130,-0.003956,0.058758
4,(rolls/buns),(UHT-milk),0.110005,0.021386,0.001804,0.016403,0.767013,1.0,-0.000548,0.994934,-0.254457,0.013925,-0.005092,0.050389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,"(yogurt, whole milk)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,1.0,0.000797,1.082219,0.548014,0.020992,0.075973,0.078050
1234,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,1.0,0.000563,1.131541,0.384877,0.009065,0.116250,0.132562
1235,(whole milk),"(yogurt, sausage)",0.157923,0.005748,0.001470,0.009310,1.619866,1.0,0.000563,1.003596,0.454430,0.009065,0.003583,0.132562
1236,(sausage),"(yogurt, whole milk)",0.060349,0.011161,0.001470,0.024363,2.182917,1.0,0.000797,1.013532,0.576701,0.020992,0.013351,0.078050


In [278]:
top_rules = rules.sort_values('lift', ascending=False).head(50)
top_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1233,"(yogurt, whole milk)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,1.0,0.000797,1.082219,0.548014,0.020992,0.075973,0.078050
1236,(sausage),"(yogurt, whole milk)",0.060349,0.011161,0.001470,0.024363,2.182917,1.0,0.000797,1.013532,0.576701,0.020992,0.013351,0.078050
1232,"(sausage, whole milk)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,1.0,0.000701,1.093681,0.481231,0.015748,0.085657,0.090650
1237,(yogurt),"(sausage, whole milk)",0.085879,0.008955,0.001470,0.017121,1.911760,1.0,0.000701,1.008307,0.521727,0.015748,0.008239,0.090650
475,(citrus fruit),(specialty chocolate),0.053131,0.015973,0.001403,0.026415,1.653762,1.0,0.000555,1.010726,0.417500,0.020731,0.010612,0.057141
474,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,1.0,0.000555,1.038081,0.401735,0.020731,0.036684,0.057141
1234,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,1.0,0.000563,1.131541,0.384877,0.009065,0.116250,0.132562
1235,(whole milk),"(yogurt, sausage)",0.157923,0.005748,0.001470,0.009310,1.619866,1.0,0.000563,1.003596,0.454430,0.009065,0.003583,0.132562
619,(tropical fruit),(flour),0.067767,0.009757,0.001069,0.015779,1.617141,1.0,0.000408,1.006118,0.409366,0.013986,0.006081,0.062684
618,(flour),(tropical fruit),0.009757,0.067767,0.001069,0.109589,1.617141,1.0,0.000408,1.046969,0.385385,0.013986,0.044862,0.062684


In [279]:
print(top_rules['antecedents'].head())
print(type(top_rules['antecedents'].iloc[0]))

1233     (yogurt, whole milk)
1236                (sausage)
1232    (sausage, whole milk)
1237                 (yogurt)
475            (citrus fruit)
Name: antecedents, dtype: object
<class 'frozenset'>


In [280]:
def clean_antecedent(x):
    if pd.isna(x):
        return ""
    # 집합/리스트/튜플이면 join
    if isinstance(x, (set, frozenset, list, tuple)):
        return ', '.join(map(str, x))
    # 문자열이면 양옆 괄호/중괄호/공백/따옴표 제거
    s = str(x).strip("(){} []'")
    # 쉼표 뒤쪽 공백도 정리
    return ', '.join([i.strip(" '\"") for i in s.split(',') if i.strip()])

top_rules['antecedents'] = top_rules['antecedents'].apply(clean_antecedent)
top_rules['consequents'] = top_rules['consequents'].apply(clean_antecedent)
print(top_rules['antecedents'].head())
print(type(top_rules['antecedents'].iloc[0]))

1233     yogurt, whole milk
1236                sausage
1232    sausage, whole milk
1237                 yogurt
475            citrus fruit
Name: antecedents, dtype: object
<class 'str'>


In [281]:
save_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top50_item_association_rules.csv"))
top_rules.to_csv(save_path, index=False, encoding='utf-8-sig')

In [282]:
print(df.columns)

Index(['Member_number', 'Date', 'itemDescription', 'Year', 'Month', 'Day',
       'Weekday', 'Season', 'YearMonth', 'WeekOfYear', 'Quarter',
       'IsMonthStart', 'IsMonthEnd', 'IsWeekend', 'DayOfYear', 'IsHoliday',
       'PrevPurchaseDelta', 'ItemCategory', 'NumCategories', 'MainCategory',
       'CategoryPurchaseDelta'],
      dtype='object')


In [283]:
# 카테고리 단위 연관분석
transactions_category = df.groupby(['Member_number', 'Date'])['ItemCategory'].apply(list).tolist()

In [284]:
# 1. 트랜잭션 리스트 생성 (카테고리 단위)
transactions_category = df.groupby(['Member_number', 'Date'])['ItemCategory'].apply(list).tolist()

# 2. One-hot 인코딩
te = TransactionEncoder()
te_ary = te.fit(transactions_category).transform(transactions_category)
df_cat_matrix = pd.DataFrame(te_ary, columns=te.columns_)

# 3. apriori로 빈발 카테고리셋 추출
frequent_catsets = apriori(df_cat_matrix, min_support=0.001, use_colnames=True)
print("빈발 카테고리셋 개수:", len(frequent_catsets))

# 4. association_rules로 규칙 추출
rules_cat = association_rules(frequent_catsets, metric="confidence", min_threshold=0.001)
print("연관규칙 개수:", len(rules_cat))

# 5. 문자열로 전처리 (frozenset → 'Bakery, Dairy' 등)
def set_to_str(x):
    if isinstance(x, (set, frozenset, list, tuple)):
        return ', '.join(sorted(map(str, x)))
    else:
        return str(x)

rules_cat['antecedents'] = rules_cat['antecedents'].apply(set_to_str)
rules_cat['consequents'] = rules_cat['consequents'].apply(set_to_str)

# 6. 상위 20개 lift 기준 추출
top20_cat = rules_cat.sort_values('lift', ascending=False).head(20)

# 7. csv 저장 (Tableau에서 사용 가능)
save_cat_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top20_cat_association_rules.csv"))
top20_cat.to_csv(save_cat_path, index=False, encoding='utf-8-sig')


빈발 카테고리셋 개수: 374
연관규칙 개수: 2126


In [285]:
# 복합 관점(품목+카테고리 조합 등)
df['ItemCatCombo'] = df['ItemCategory'] + ':' + df['itemDescription']
transactions_combo = df.groupby(['Member_number', 'Date'])['ItemCatCombo'].apply(list).tolist()

In [286]:
te = TransactionEncoder()
te_ary = te.fit(transactions_combo).transform(transactions_combo)
df_combo_matrix = pd.DataFrame(te_ary, columns=te.columns_)

In [287]:
frequent_combosets = apriori(df_combo_matrix, min_support=0.001, use_colnames=True)
print("빈발 복합 아이템셋 개수:", len(frequent_combosets))


빈발 복합 아이템셋 개수: 750


In [288]:
rules_combo = association_rules(frequent_combosets, metric="confidence", min_threshold=0.001)
print("연관규칙 개수:", len(rules_combo))

연관규칙 개수: 1238


In [289]:
def set_to_str(x):
    if isinstance(x, (set, frozenset, list, tuple)):
        return ', '.join(sorted(map(str, x)))
    else:
        return str(x)

rules_combo['antecedents'] = rules_combo['antecedents'].apply(set_to_str)
rules_combo['consequents'] = rules_combo['consequents'].apply(set_to_str)
top20_combo = rules_combo.sort_values('lift', ascending=False).head(20)

save_com_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top20_com_association_rules.csv"))
top20_combo.to_csv(save_com_path, index=False, encoding='utf-8-sig')

In [290]:
all_weekday_rules = []

for wd in df['Weekday'].unique():
    df_wd = df[df['Weekday'] == wd]
    transactions_wd = df_wd.groupby(['Member_number', 'Date'])['ItemCatCombo'].apply(list).tolist()
    if not transactions_wd: continue

    te = TransactionEncoder()
    te_ary = te.fit(transactions_wd).transform(transactions_wd)
    df_matrix = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_sets = apriori(df_matrix, min_support=0.001, use_colnames=True)
    if frequent_sets.empty: continue

    rules = association_rules(frequent_sets, metric="confidence", min_threshold=0.001)
    if rules.empty: continue

    # frozenset/string 변환
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))

    top20 = rules.sort_values('lift', ascending=False).head(20)
    top20['Weekday'] = wd    # ★ 요일 컬럼 추가
    all_weekday_rules.append(top20)

# 하나의 DataFrame으로 합침
all_weekday_rules_df = pd.concat(all_weekday_rules, ignore_index=True)
all_weekday_rules_df = all_weekday_rules_df.sort_values('lift', ascending=False).head(20)
save_weekday_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top20_weekday_association_rules.csv"))
all_weekday_rules_df.to_csv(save_weekday_path, index=False, encoding='utf-8-sig')

In [291]:
all_month_rules = []

for m in df['Month'].unique():
    df_m = df[df['Month'] == m]
    transactions_m = df_m.groupby(['Member_number', 'Date'])['ItemCatCombo'].apply(list).tolist()
    if not transactions_m: continue

    te = TransactionEncoder()
    te_ary = te.fit(transactions_m).transform(transactions_m)
    df_matrix = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_sets = apriori(df_matrix, min_support=0.001, use_colnames=True)
    if frequent_sets.empty: continue

    rules = association_rules(frequent_sets, metric="confidence", min_threshold=0.001)
    if rules.empty: continue

    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))

    top20 = rules.sort_values('lift', ascending=False).head(20)
    top20['Month'] = m     # ★ 월 컬럼 추가
    all_month_rules.append(top20)

all_month_rules_df = pd.concat(all_month_rules, ignore_index=True)
all_month_rules_df = all_month_rules_df.sort_values('lift', ascending=False).head(20)
save_months_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top20_months_association_rules.csv"))
all_month_rules_df.to_csv(save_months_path, index=False, encoding='utf-8-sig')


In [292]:
all_season_rules = []

for s in df['Season'].unique():
    df_s = df[df['Season'] == s]
    transactions_s = df_s.groupby(['Member_number', 'Date'])['ItemCatCombo'].apply(list).tolist()
    if not transactions_s: continue

    te = TransactionEncoder()
    te_ary = te.fit(transactions_s).transform(transactions_s)
    df_matrix = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_sets = apriori(df_matrix, min_support=0.001, use_colnames=True)
    if frequent_sets.empty: continue

    rules = association_rules(frequent_sets, metric="confidence", min_threshold=0.001)
    if rules.empty: continue

    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(sorted(map(str, x))) if isinstance(x, (set, frozenset, list, tuple)) else str(x))

    top20 = rules.sort_values('lift', ascending=False).head(20)
    top20['Season'] = s     # ★ 계절 컬럼 추가
    all_season_rules.append(top20)

all_season_rules_df = pd.concat(all_season_rules, ignore_index=True)
all_season_rules_df = all_season_rules_df.sort_values('lift', ascending=False).head(20)
save_seasons_path = os.path.abspath(os.path.join(os.getcwd(), "..", "data", "top20_seasons_association_rules.csv"))
all_season_rules_df.to_csv(save_seasons_path, index=False, encoding='utf-8-sig')
